## comparing the model via MSE and MAE on the testing split of the data 


In [1]:
from models.model_tools import load_best_model, evaluate_models
from models.unet_beamformer import UNetBeamformer
from trainer.trainer import Trainer
import torch
import argparse
from dataset import CustomDataset
import pandas as pd
import os
from models.transformer_cnn import TransformerCNN
from models.model_tools import get_best_checkpoint_path, load_best_model

parser = argparse.ArgumentParser()
parser.add_argument("--data", type=str,default = 'wrist_tofc')
parser.add_argument("--save", type=str,default = 'comparison_results')
parser.add_argument("--epochs", type=int,default=30)
parser.add_argument("--bs", type=int,default=8)
parser.add_argument("--lr", type=float,default=1e-3)
parser.add_argument("--run", type=int,default=1)
parser.add_argument("--num_workers", type=int,default=1)
parser.add_argument("--loss", type=str, default= 'MAE')
args = parser.parse_args([])

device = "cuda" if torch.cuda.is_available() else "cpu"




best_UNET_MSE = load_best_model(UNetBeamformer, 'Results/UNET_30_MAE_Results/chkpt/iter_1').to(device)
best_UNET_MAE = load_best_model(UNetBeamformer, 'Results/UNET_30_MSE_Results/chkpt/iter_1').to(device)
best_TransformerCNN_MSE = load_best_model(TransformerCNN, 'Results/TransformerCNN_30_MAE_Results/chkpt/iter_1').to(device)
best_TransformerCNN_MAE = load_best_model(TransformerCNN, 'Results/TransformerCNN_30_MSE_Results/chkpt/iter_1').to(device)

DATASET = CustomDataset(args.data)

T = Trainer(TransformerCNN, DATASET, args)
test_loader = T.valid_data
test_set = T.valid_set

UNET_MSE_mse_loss, UNET_MSE_mae_loss = evaluate_models(best_UNET_MSE, test_loader, test_set)
UNET_MAE_mse_loss, UNET_MAE_mae_loss = evaluate_models(best_UNET_MAE, test_loader, test_set)
TransformerCNN_MSE_mse_loss, TransformerCNN_MSE_mae_loss = evaluate_models(best_TransformerCNN_MSE, test_loader, test_set)
TransformerCNN_MAE_mse_loss, TransformerCNN_MAE_mae_loss = evaluate_models(best_TransformerCNN_MAE, test_loader, test_set)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Model': ['UNET_MSE', 'UNET_MAE', 'TransformerCNN_MSE', 'TransformerCNN_MAE'],
    'MSE': [UNET_MSE_mse_loss, UNET_MAE_mse_loss, TransformerCNN_MSE_mse_loss, TransformerCNN_MAE_mse_loss],
    'MAE': [UNET_MSE_mae_loss, UNET_MAE_mae_loss, TransformerCNN_MSE_mae_loss, TransformerCNN_MAE_mae_loss]
})

# Set 'Model' as the index for better readability
results_df.set_index('Model', inplace=True)

# Display the results
print(results_df)


/home/jeremy/Documents/code/Medical_Image_Reconstruction/models/model_tools.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_c

Loaded best model from Results/UNET_30_MAE_Results/chkpt/iter_1/best.pt
Loaded best model from Results/UNET_30_MSE_Results/chkpt/iter_1/best.pt
Loaded best model from Results/TransformerCNN_30_MAE_Results/chkpt/iter_1/best.pt
Loaded best model from Results/TransformerCNN_30_MSE_Results/chkpt/iter_1/best.pt


/home/jeremy/Documents/code/Medical_Image_Reconstruction/models/model_tools.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


                          MSE       MAE
Model                                  
UNET_MSE            55.826221  5.579086
UNET_MAE            70.324293  6.453071
TransformerCNN_MSE  57.217969  5.677939
TransformerCNN_MAE  74.578333  6.660878


In [2]:
results_df

,MSE,MAE
Model,,
UNET_MSE,55.826221,5.579086
UNET_MAE,70.324293,6.453071
TransformerCNN_MSE,57.217969,5.677939
TransformerCNN_MAE,74.578333,6.660878


In [3]:
# Calculate average for MSE-trained models
mse_trained_avg = results_df.loc[['UNET_MSE', 'TransformerCNN_MSE']].mean()
results_df.loc['avg MSE_trained_model'] = mse_trained_avg

# Calculate average for MAE-trained models
mae_trained_avg = results_df.loc[['UNET_MAE', 'TransformerCNN_MAE']].mean()
results_df.loc['avg MAE_trained_model'] = mae_trained_avg

# Calculate average for UNET models
unet_avg = results_df.loc[['UNET_MSE', 'UNET_MAE']].mean()
results_df.loc['UNET_avg'] = unet_avg

# Calculate average for TransformerCNN models
transformer_cnn_avg = results_df.loc[['TransformerCNN_MSE', 'TransformerCNN_MAE']].mean()
results_df.loc['TransformerCNN_avg'] = transformer_cnn_avg

# Display the updated results
print(results_df)


                             MSE       MAE
Model                                     
UNET_MSE               55.826221  5.579086
UNET_MAE               70.324293  6.453071
TransformerCNN_MSE     57.217969  5.677939
TransformerCNN_MAE     74.578333  6.660878
avg MSE_trained_model  56.522095  5.628512
avg MAE_trained_model  72.451313  6.556975
UNET_avg               63.075257  6.016079
TransformerCNN_avg     65.898151  6.169408


In [4]:
results_df

,MSE,MAE
Model,,
UNET_MSE,55.826221,5.579086
UNET_MAE,70.324293,6.453071
TransformerCNN_MSE,57.217969,5.677939
TransformerCNN_MAE,74.578333,6.660878
avg MSE_trained_model,56.522095,5.628512
avg MAE_trained_model,72.451313,6.556975
UNET_avg,63.075257,6.016079
TransformerCNN_avg,65.898151,6.169408
